In [1]:
import torch
from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim
TB_writer = SummaryWriter()
from GAN_model.resnetGAN import *
from GAN_model.ProteoGAN import *
from utils.dataloader import *
from utils.torch_utils import *
from utils.metrics import *

In [2]:
args_run_name = 'ProteoGAN_GFP_MSA'
# args; TODO: move these to parser for script
args_batch_size=16; args_seed=42; args_eval_n_seq = 500
args_epoch = 300
# GAN specific param
args_z_dim = 128; args_dim = 256
# optimizer specific param
args_lr_Gen = 1e-4; args_lr_Disc = 8e-5; args_loss = 'hinge'
# train discriminator more initially
# according to https://livebook.manning.com/book/gans-in-action/chapter-5/185
args_disc_iters_init = 2; args_disc_iters = 1; args_disc_iters_init_epoch = 50
# make dir
os.makedirs('checkpoints/'+args_run_name, exist_ok=True)
os.makedirs('out/'+args_run_name+'/plot_out/', exist_ok=True)
os.makedirs('out/'+args_run_name+'/seq_out/', exist_ok=True)

In [3]:
seed_everything(args_seed)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
# keep dataset in cpu
MSA_data = MSA_dataset('GFP_data/gfp_MSA_clean.fasta', 
    torch.device('cpu'), pseudo_MSA=False) 

### Creating data sets

In [4]:
# creating data set with train/val split
train_len = int(MSA_data.length*0.85-MSA_data.length)
val_len = MSA_data.length - train_len
train_data, val_data = torch.utils.data.random_split(
    MSA_data, [train_len, val_len], 
    generator=torch.Generator().manual_seed(args_seed))
# decoding train and validation sequences for evaluation
val_seqs = [decode_one_seq(torch.argmax(data, dim=-1).detach().cpu()) 
            for data in val_data]
tr_seqs = [decode_one_seq(torch.argmax(data, dim=-1).detach().cpu()) 
           for data in train_data]
# creating dataloader for training data
loader = torch.utils.data.DataLoader(
    train_data, batch_size=args_batch_size, shuffle=True,
    num_workers=1, pin_memory=True)
# fixed parameters
n_chars = 21; seq_len = len(val_seqs[0])

### Evaluating positive and negative control sequences to obtain baseline metrics
Positive: sample of real sequences and simulates a perfect model

Negative: a sample that simulates the worst possible model for each metric: \
    (constant sequence for MMD, repeated sequences for diversity measures).


In [5]:
pos_ctrl_seqs = np.random.choice(tr_seqs, size=len(val_seqs), replace=False)
# constant sequences for MMD
neg_ctrl_seqs_MMD = []
for _ in range(len(val_seqs)):
    rand_aa_idx = np.random.randint(0, high=19)
    rand_seq = decode_one_seq(
        np.random.randint(rand_aa_idx, high=rand_aa_idx+2, size=seq_len))
    neg_ctrl_seqs_MMD.append(rand_seq)
# repeated sequences for diversity measures
rand_seq = decode_one_seq(np.random.randint(0, high=21, size=seq_len))
neg_ctrl_seqs_diversity = [rand_seq for _ in range(len(val_seqs))]
# report baseline
MMD_pos = mmd(seq1=pos_ctrl_seqs, seq2=val_seqs)
MMD_neg = mmd(seq1=neg_ctrl_seqs_MMD, seq2=val_seqs)
Entropy_pos = abs(entropy(seq1=pos_ctrl_seqs, seq2=val_seqs))
Entropy_neg = abs(entropy(seq1=neg_ctrl_seqs_diversity, seq2=val_seqs))
Distance_pos = abs(distance(seq1=pos_ctrl_seqs, seq2=val_seqs))
Distance_neg = abs(distance(seq1=neg_ctrl_seqs_diversity, seq2=val_seqs))
Pearson_cor_pos = pearson_cor(seq1=pos_ctrl_seqs, seq2=val_seqs, random_n=1000)[1].item()

C:\Users\canal\AppData\Roaming\Python\Python38\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\canal\AppData\Roaming\Python\Python38\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [6]:
## Pearson corr higher the better, all other lower the better
print('>>> MMD | abs(D_Entropy) | abs(D_Distance) | Pearson Corr' )
print(f'>>> For postivie control:\n'
      f'{MMD_pos:.5} , {Entropy_pos:.5} , {Distance_pos:.5} , {Pearson_cor_pos:.5}')
print(f'>>> For negative control:\n'
      f'{MMD_neg:.5} , {Entropy_neg:.5} , {Distance_neg:.5} , 0.000')

>>> MMD | abs(D_Entropy) | abs(D_Distance) | Pearson Corr
>>> For postivie control:
0.14012 , 0.008014 , 0.076579 , nan
>>> For negative control:
0.72962 , 0.15464 , 1.0896 , 0.000


In [7]:
def save_checkpoints(discriminator, generator, optim_D, optim_G,  
               scheduler_D, scheduler_G, epoch):
    torch.save({
        'epoch': epoch,
        'disc_state_dict': discriminator.state_dict(),
        'gen_state_dict': generator.state_dict(),
        'optim_D_state_dict': optim_D.state_dict(),
        'optim_G_state_dict': optim_G.state_dict(),
        'scheduler_D_state_dict': scheduler_D.state_dict(),
        'scheduler_G_state_dict': scheduler_G.state_dict(),
    }, os.path.join('checkpoints/'+args_run_name, 'epoch_{}'.format(epoch)))
    return

def load_checkpoints(discriminator, generator, optim_D, optim_G,
                     scheduler_D, scheduler_G, epoch):
    checkpoints = torch.load(
        os.path.join('checkpoints'+args_run_name, 'epoch_{}'.format(epoch)))  
    discriminator.load_state_dict(checkpoints['disc_state_dict'])
    generator.load_state_dict(checkpoints['gen_state_dict'])
    optim_D.load_state_dict(checkpoints['optim_D_state_dict'])
    optim_G.load_state_dict(checkpoints['optim_G_state_dict'])
    scheduler_D.load_state_dict(checkpoints['scheduler_D_state_dict'])
    scheduler_G.load_state_dict(checkpoints['scheduler_G_state_dict'])
    return

## Init model

In [8]:
# init model and optimizer

Disc = Discriminator(args_dim, seq_len=seq_len, n_chars=n_chars).to(device)
Gen = Generator(args_dim, seq_len=seq_len, n_chars=n_chars, 
                z_dim=args_z_dim).to(device)

# Disc = ResNetSN_Discriminator(args_dim, seq_len, n_chars).to(device)
# Gen = ResNetSN_Generator(args_dim, seq_len, n_chars, args_z_dim).to(device)

optim_disc = optim.Adam(filter(lambda p: p.requires_grad, Disc.parameters()),
                        lr=args_lr_Disc, betas=(0.0, 0.9))
optim_gen = optim.Adam(Gen.parameters(), lr=args_lr_Gen, betas=(0.0, 0.9))
# use an exponentially decaying learning rate
scheduler_d = optim.lr_scheduler.ExponentialLR(optim_disc, gamma=0.99)
scheduler_g = optim.lr_scheduler.ExponentialLR(optim_gen, gamma=0.99)

#load_checkpoints(Disc, Gen, optim_disc, optim_gen, scheduler_d, scheduler_g, 0)
#save_checkpoints(Disc, Gen, optim_disc, optim_gen, scheduler_d, scheduler_g, 0)

In [9]:
def sample_generated(generator, epoch, run_name, n_seq=500):
    # random sampling n_seq noise, batched 100 noise at a time
    # return sampled n_seq sequences, also saving them to fasta file
    gen_seqs = []
    max_batch_size = 100
    for _ in range(n_seq//max_batch_size):
        z = torch.randn(max_batch_size, args_z_dim, device=device)
        gen_probs = generator(z)
        batched_seq = [decode_one_seq(
            gen_prob.argmax(-1).cpu().numpy()) for gen_prob in gen_probs]
        gen_seqs += batched_seq
        torch.cuda.empty_cache()
    # saving seqs in fasta
    fasta_name = 'out/' + run_name + '/seq_out/epoch_' + str(epoch) + '.fasta'
    outfile = open(fasta_name, 'w')
    for i, seq in enumerate(gen_seqs):
        seq_id = 'Epoch'+str(epoch)+'_seq'+str(i)
        outfile.write('>' + seq_id + '\n')
        outfile.write(seq + '\n')
    return gen_seqs


def write_eval_TB(writer, MMD, Entropy, Distance, Pearson, epoch):
    writer.add_scalar(args_run_name+'/MMD',  MMD, epoch)
    writer.add_scalar(args_run_name+'/abs(D_Entropy)', Entropy, epoch)
    writer.add_scalar(args_run_name+'/abs(D_Distance)', Distance, epoch)
    writer.add_scalar(args_run_name+'/Pearson Correlation', Pearson, epoch)
    return

def evaluate(epoch):
    # eval mode
    Gen.eval()
    # sample args_eval_n_seq sequences
    gen_seqs = sample_generated(Gen, epoch, args_run_name, n_seq=args_eval_n_seq)
    MMD = mmd(seq1=gen_seqs, seq2=val_seqs)
    Entropy = abs(entropy(seq1=gen_seqs, seq2=val_seqs))
    Distance = abs(distance(seq1=gen_seqs, seq2=val_seqs))
    Pearson = pearson_cor(
        seq1=gen_seqs, seq2=val_seqs, random_n=args_eval_n_seq)[1].item()
    print(f'#========== evaluating for epoch ' f'{epoch} ' f'==========#')
    print('>>> MMD | abs(D_Entropy) | abs(D_Distance) | Pearson Corr \n'
          f'{MMD:.5} , {Entropy:.5} , {Distance:.5} , {Pearson:.5}')
    # save the gap distribution plot
    plot_gap_dist(seqs=gen_seqs, run_name=args_run_name, epoch=epoch)
    write_eval_TB(TB_writer, MMD, Entropy, Distance, Pearson, epoch)
    # revert to train mode
    Gen.train()
    return


In [10]:
def train(epoch):
    for batch_idx, data in enumerate(loader):
        if data.size()[0] != args_batch_size:
            continue
        else:
            data = data.to(device)
        # train discriminator more initially
        if epoch <= args_disc_iters_init_epoch: 
            disc_iters = args_disc_iters_init
        else:
            disc_iters = args_disc_iters
        # update discriminator
        for _ in range(disc_iters):
            z = torch.randn(args_batch_size, args_z_dim, device=device)
            optim_disc.zero_grad()
            optim_gen.zero_grad()
            real_D = Disc(data)
            fake_D = Disc(Gen(z))
            if args_loss == 'hinge':
                disc_loss = nn.ReLU()(1.0 - real_D).mean() + \
                nn.ReLU()(1.0 + fake_D).mean()
            disc_loss.backward()
            optim_disc.step()
        # record output for both fake and real from discriminator
        real_prob_avg = real_D.mean()
        fake_prob_avg = fake_D.mean()
        
        # update generator
        z = torch.randn(args_batch_size, args_z_dim, device=device)
        optim_disc.zero_grad()
        optim_gen.zero_grad()
        if args_loss == 'hinge' or args_loss == 'wasserstein':
            gen_loss = -Disc(Gen(z)).mean()
        gen_loss.backward()
        optim_gen.step()
        
        # write to Tensor board
        iter = len(loader)*epoch + batch_idx
        TB_writer.add_scalars(
            args_run_name+'/GAN loss', {
            'Disc_loss': disc_loss,
            'Gen_loss': gen_loss,
        }, iter)
        TB_writer.add_scalars(
            args_run_name+'/Discriminator out', {
            'Real prob': real_prob_avg,
            'Fake prob': fake_prob_avg,
        }, iter)
        if batch_idx % 60 == 0:
            print('disc loss', disc_loss.item(), 'gen loss', gen_loss.item())
    # update lr scheduler
    scheduler_d.step()
    scheduler_g.step()
    return


In [11]:
for epoch in range(args_epoch):
    train(epoch)
    save_checkpoints(Disc, Gen, optim_disc, optim_gen, 
                     scheduler_d, scheduler_g, epoch)
    if epoch != 0:
        os.remove(os.path.join('checkpoints/'+args_run_name, 
                               'epoch_{}'.format(epoch-1)))
    # eval every 2 epoch
    if epoch % 2 == 0:
        evaluate(epoch)

disc loss 1.9671287536621094 gen loss 0.05788945406675339
#========== evaluating for epoch 0 ==========#
>>> MMD | abs(D_Entropy) | abs(D_Distance) | Pearson Corr 
0.66801 , 0.071965 , 0.85698 , nan
# ================== Sampling 500 sequences at epoch 0 ================== #
The loci with percentage of gap >= 95% in the cleaned MSA: 
 []
Their percentage gap are:
 []
# ====================================================== #
disc loss 1.538091778755188 gen loss 0.05099334195256233
disc loss 1.0147287845611572 gen loss 0.11223749816417694


C:\Users\canal\AppData\Roaming\Python\Python38\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\canal\AppData\Roaming\Python\Python38\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


#========== evaluating for epoch 2 ==========#
>>> MMD | abs(D_Entropy) | abs(D_Distance) | Pearson Corr 
0.6677 , 0.070777 , 0.85699 , nan
# ================== Sampling 500 sequences at epoch 2 ================== #
The loci with percentage of gap >= 95% in the cleaned MSA: 
 []
Their percentage gap are:
 []
# ====================================================== #
disc loss 0.6638278365135193 gen loss 0.38347333669662476
disc loss 0.17272460460662842 gen loss 0.8952022790908813


C:\Users\canal\AppData\Roaming\Python\Python38\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\canal\AppData\Roaming\Python\Python38\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


#========== evaluating for epoch 4 ==========#
>>> MMD | abs(D_Entropy) | abs(D_Distance) | Pearson Corr 
0.66616 , 0.07162 , 0.85694 , nan
# ================== Sampling 500 sequences at epoch 4 ================== #
The loci with percentage of gap >= 95% in the cleaned MSA: 
 []
Their percentage gap are:
 []
# ====================================================== #
disc loss 0.006121009588241577 gen loss 1.049099087715149
disc loss 0.033028941601514816 gen loss 1.1891436576843262


...

#========== evaluating for epoch 294 ==========#
>>> MMD | abs(D_Entropy) | abs(D_Distance) | Pearson Corr 
0.30758 , 0.064864 , 0.043487 , nan
# ================== Sampling 500 sequences at epoch 294 ================== #
The loci with percentage of gap >= 95% in the cleaned MSA: 
 [0, 1, 2, 3, 78, 154, 155, 228, 229, 230, 231, 233, 235, 236]
Their percentage gap are:
 [100.0, 99.2, 98.2, 96.6, 100.0, 95.2, 95.8, 99.6, 99.4, 96.6, 98.8, 99.0, 99.0, 99.6]
# ====================================================== #
disc loss 0.9368871450424194 gen loss 0.856451690196991
disc loss 0.5387342572212219 gen loss 0.8735041618347168


C:\Users\canal\AppData\Roaming\Python\Python38\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\canal\AppData\Roaming\Python\Python38\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


#========== evaluating for epoch 296 ==========#
>>> MMD | abs(D_Entropy) | abs(D_Distance) | Pearson Corr 
0.30532 , 0.063738 , 0.064296 , nan
# ================== Sampling 500 sequences at epoch 296 ================== #
The loci with percentage of gap >= 95% in the cleaned MSA: 
 [0, 1, 2, 3, 78, 153, 154, 155, 228, 229, 230, 231, 233, 234, 235, 236]
Their percentage gap are:
 [100.0, 98.6, 99.0, 96.4, 100.0, 97.4, 96.8, 95.2, 98.2, 99.8, 97.8, 99.2, 99.8, 95.4, 99.2, 99.6]
# ====================================================== #
disc loss 0.7996231317520142 gen loss 0.8229312896728516
disc loss 0.6892539858818054 gen loss 0.7856919765472412


C:\Users\canal\AppData\Roaming\Python\Python38\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\canal\AppData\Roaming\Python\Python38\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


#========== evaluating for epoch 298 ==========#
>>> MMD | abs(D_Entropy) | abs(D_Distance) | Pearson Corr 
0.3063 , 0.063121 , 0.077253 , nan
# ================== Sampling 500 sequences at epoch 298 ================== #
The loci with percentage of gap >= 95% in the cleaned MSA: 
 [0, 1, 2, 3, 78, 153, 154, 155, 228, 229, 230, 231, 233, 234, 235, 236]
Their percentage gap are:
 [100.0, 99.0, 99.8, 97.0, 99.6, 97.2, 96.8, 96.8, 99.6, 98.6, 97.0, 99.8, 99.8, 95.4, 99.4, 99.8]
# ====================================================== #
disc loss 0.5317018032073975 gen loss 0.8790132999420166
